In [ ]:
import os
os.chdir("../")

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset
import torch
import torch.nn as nn
from src import T5ModelForTableReasoning, T5ModelForTableCellHighlighting

In [ ]:
from data import WikiTQHighlightedCellsDataset

In [ ]:
import json
from utils import process_config
with open("configs/wiki_tq_clustering_and_highlighting/tapex.json", "rb") as f:
    config = json.load(f)
config = process_config(config)

In [ ]:
dataset = load_dataset("wikitablequestions")

In [ ]:
test_dataset = WikiTQHighlightedCellsDataset(dataset = dataset, config = config, data_type = "test")

In [ ]:
tokenizer = test_dataset.tokenizer

In [ ]:
original_tables = test_dataset.table

In [ ]:
from tqdm import tqdm

In [ ]:
import numpy as np
original_tables_num_column = np.array([len(x.columns.values) for x in original_tables])

In [ ]:
original_tables_num_column[0]

In [ ]:
perturbed_tables = []

In [ ]:
import pandas as pd

In [ ]:
def perturb_table(table, text_input, idx = None):

    tokenized_len = tokenizer(table = table, text_input = text_input, add_special_tokens = config.tokenizer.add_special_tokens, 
                 return_tensors = config.tokenizer.return_tensors)["input_ids"][0].shape[0]
    
    if tokenized_len > 1000:
        return table
    
    else:
        num_columns = len(table.columns.values)
        indices = np.where(original_tables_num_column == num_columns)[0]
        
        if len(indices) == 1:
            return table
        else:
            indices = indices[indices != idx]
            table_index = np.random.choice(indices)
            num_rows = len(original_tables[table_index])

            if tokenized_len > 900:
                if num_rows < 1:
                    return table
                else:
                    random_index = np.random.randint(0, num_rows)
                    insert_index = np.random.randint(0, len(table))
                    
                    new_row = pd.DataFrame(original_tables[table_index].iloc[random_index]).T
                    new_row.columns = table.columns.values
                    perturbed_table = pd.concat([table.loc[:insert_index], new_row, table.loc[insert_index:]]).reset_index(drop=True)

            elif tokenized_len > 800:
                if num_rows < 2:
                    return table
                else:
                    random_index = np.random.randint(0, num_rows, size = 2)
                    insert_index = np.random.randint(0, len(table))
                    
                    new_row = pd.DataFrame(original_tables[table_index].iloc[random_index])
                    new_row.columns = table.columns.values
                    perturbed_table = pd.concat([table.loc[:insert_index], new_row, table.loc[insert_index:]]).reset_index(drop=True)
            elif tokenized_len > 500:
                if num_rows < 3:
                    return table
                else:
                    random_index = np.random.randint(0, num_rows, size = 3)
                    insert_index = np.random.randint(0, len(table))
                    
                    new_row = pd.DataFrame(original_tables[table_index].iloc[random_index])
                    new_row.columns = table.columns.values
                    perturbed_table = pd.concat([table.loc[:insert_index], new_row, table.loc[insert_index:]]).reset_index(drop=True)

            else:
                if num_rows < 5:
                    return table
                else:
                    random_index = np.random.randint(0, num_rows, size = 5)
                    insert_index = np.random.randint(0, len(table))                    
                    
                    new_row = pd.DataFrame(original_tables[table_index].iloc[random_index])
                    new_row.columns = table.columns.values

                    perturbed_table = pd.concat([table.loc[:insert_index], new_row, table.loc[insert_index:]]).reset_index(drop=True)
            
            return perturbed_table

In [ ]:
for i, (table, text) in tqdm(enumerate(zip(original_tables, test_dataset.text_input)), position = 0, leave = True, total = len(test_dataset)):

    # if i == 1:
    perturbed_table = perturb_table(table=table, text_input=text, idx = i)
    perturbed_tables.append(perturbed_table)
    # display(table)
    # display(perturbed_table)
    # break

In [ ]:
import pickle
with open("datasets/wiki_tq_test_perturbed_tables.pkl", "wb") as f:
    pickle.dump(perturbed_tables, f)

In [ ]:
with open("configs/wiki_tq_reasoning/t5.json", "rb") as f:
    reason_generation_config = json.load(f)
reason_generation_config = process_config(reason_generation_config)

In [ ]:
reason_generation_config.training.training_type = "table_question_answering"

In [ ]:
test_dataset.table = perturbed_tables
test_dataset.config = reason_generation_config

In [ ]:
model = T5ModelForTableReasoning(reason_generation_config)
model.load_state_dict(torch.load("Flan T5 Reason Generator/epoch=50.pt", map_location = "cpu"))

In [ ]:
model.to("cuda:0")

In [ ]:
for i, table in enumerate(perturbed_tables):
    table_column_names = table.columns.values.tolist()
    table_content_values = table.values.tolist()
    table = "[HEADER] " + " | ".join(table_column_names)
    for row_id, row in enumerate(table_content_values):
        table += f" [ROW] {row_id}: " + " | ".join(row)
    
    test_dataset.table[i] = table

In [ ]:
reason_generations = []

In [ ]:
from transformers import AutoTokenizer
reason_tokenizer = AutoTokenizer.from_pretrained(reason_generation_config.tokenizer.tokenizer_path)
if "bos_token" not in list(reason_tokenizer.special_tokens_map.keys()):
    reason_tokenizer.add_special_tokens({"bos_token": "<s>"})

if "pad_token" not in list(reason_tokenizer.special_tokens_map.keys()):
    reason_tokenizer.add_special_tokens({"pad_token": reason_tokenizer.special_tokens_map["eos_token"]})

if "sep_token" not in list(reason_tokenizer.special_tokens_map.keys()):
    reason_tokenizer.add_special_tokens({"sep_token": reason_tokenizer.special_tokens_map["eos_token"]})

if "mask_token" not in list(reason_tokenizer.special_tokens_map.keys()):
    reason_tokenizer.add_special_tokens({"mask_token": reason_tokenizer.special_tokens_map["eos_token"]})

test_dataset.tokenizer = reason_tokenizer

In [ ]:
test_dataset.config.data.use_highlighted_cells = False

In [ ]:
from torch.utils.data import DataLoader
test_dataloader = DataLoader(test_dataset, batch_size = 24, shuffle = False, num_workers = config.system.num_workers)

In [ ]:
for i, batch in tqdm(enumerate(test_dataloader), position = 0, leave = True, total = len(test_dataloader)):

    input_ids, attention_mask, _, _, labels = batch
    predicted_ids = model.model.generate(input_ids = input_ids.to("cuda:0"), attention_mask = attention_mask.to("cuda:0"), 
                                     max_new_tokens = config.tokenizer.output_max_length, num_beams = 3, early_stopping = True).detach().cpu()

    batch_predicted_reason = test_dataset.tokenizer.batch_decode(predicted_ids, skip_special_tokens = True)

    reason_generations.extend(batch_predicted_reason)

In [ ]:
with open("datasets/wiki_tq_test_perturbed_tables_reasons.pkl", "wb") as f:
    pickle.dump(reason_generations, f)

In [ ]:
with open("configs/cell_highlighting/t5.json", "rb") as f:
    cell_highlighting_config = json.load(f)
cell_highlighting_config = process_config(cell_highlighting_config)

In [ ]:
model = T5ModelForTableCellHighlighting(cell_highlighting_config)
model.load_state_dict(torch.load("Flan T5 highlighter/epoch=2.pt", map_location="cpu"))

In [ ]:
model.to("cuda:1")

In [ ]:
with open("datasets/wiki_tq_test_perturbed_tables_reasons.pkl", "rb") as f:
    reasons_list = pickle.load(f)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(cell_highlighting_config.tokenizer.tokenizer_path)

In [ ]:
dataset = load_dataset("wikitablequestions")

In [ ]:
def predict(idx):

    reason = reasons_list[idx]

    table = perturbed_tables[idx]
    # table_column_names = table.columns.values
    # table_content_values = table.values
    # table_column_names = dataset["test"][idx]["table"]["header"]
    # table_content_values = dataset["test"][idx]["table"]["rows"]

    # table_column_names = [x.lower() for x in table_column_names]

    # table = "[HEADER] " + " | ".join(table_column_names)
    # for row_id, row in enumerate(table_content_values):
    #     row = [x.lower() for x in row]
    #     table += f" [ROW] {row_id}: " + " | ".join(row) 

    tokenized_input = tokenizer(reason, table, add_special_tokens = config.tokenizer.add_special_tokens,
                            padding = config.tokenizer.padding, truncation = config.tokenizer.truncation, 
                            max_length = config.tokenizer.input_max_length, return_tensors = config.tokenizer.return_tensors,
                            return_token_type_ids = config.tokenizer.return_token_type_ids,
                            return_attention_mask = config.tokenizer.return_attention_mask)


    output_ids = model.model.generate(input_ids = tokenized_input["input_ids"].to("cuda:1"), attention_mask = tokenized_input["attention_mask"].to("cuda:1"),
                                      max_new_tokens = config.tokenizer.output_max_length, num_beams = 3, early_stopping = True).squeeze().detach().cpu()

    predicted_cells = tokenizer.decode(output_ids, skip_special_tokens=True)

    return predicted_cells

In [ ]:
highlighted_cells_list = []

In [ ]:
for i in tqdm(range(len(reasons_list)), position=0, leave = True, total = len(reasons_list)):
    x = predict(i).split(", ")
    x = [a.strip() for a in x]
    highlighted_cells_list.append(x)

In [ ]:
with open("datasets/wiki_tq_test_highlighted_cell_perturbed_tables.pkl", "wb") as f:
    pickle.dump(highlighted_cells_list, f)

In [ ]:
column_names = [x.strip() for x in perturbed_tables[0].split("[ROW]")[0].split("[HEADER]")[1].strip().split("|")]
column_names

In [ ]:
rows = [[ele.strip() for ele in value.strip().split(":")[-1].strip().split("|")] for value in perturbed_tables[2].split("[ROW]")[1:]]

In [ ]:
perturbed_tables_df = []

In [ ]:
for pt in tqdm(perturbed_tables, position = 0, leave = True, total = len(perturbed_tables)):
    table_column_names = [x.strip() for x in pt.split("[ROW]")[0].split("[HEADER]")[1].strip().split("|")]
    table_content_values = [[ele.strip() for ele in " ".join(value.strip().split(":")[1:]).strip().split("|")] for value in pt.split("[ROW]")[1:]]

    table_df = pd.DataFrame.from_dict({str(col).lower(): [str(table_content_values[j][i]).lower() for j in range(len(table_content_values))] for i, col in enumerate(table_column_names)})

    perturbed_tables_df.append(table_df)

In [ ]:
len(perturbed_tables_df)

In [ ]:
with open("datasets/wiki_tq_test_perturbed_tables.pkl", "wb") as f:
    pickle.dump(perturbed_tables_df, f)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/tapex-large")

In [ ]:
test_dataset = dataset["test"]

In [ ]:
import pandas as pd
hard_relevance_labels = []

In [ ]:
with open("datasets/wiki_tq_test_perturbed_tables.pkl", "rb") as f:
    perturbed_tables_list = pickle.load(f)

In [ ]:
for i in tqdm(range(len(test_dataset)), position = 0, leave = True, total = len(test_dataset)):

    question = test_dataset[i]["question"]

    table_column_names = perturbed_tables_df[i].columns.values
    table_content_values = perturbed_tables_df[i].values

    highlighted_cells = highlighted_cells_list[i]

    # table = train_dataset[i]["table"]
    # table_column_names = table["header"]    
    # table_content_values = table["rows"]

    table_df = pd.DataFrame.from_dict({str(col).lower(): [str(table_content_values[j][i]).lower() for j in range(len(table_content_values))] for i, col in enumerate(table_column_names)})
    
    tokenized_input = tokenizer(table_df, question, add_special_tokens = config.tokenizer.add_special_tokens,
                            padding = config.tokenizer.padding, truncation = config.tokenizer.truncation, 
                            max_length = 960, return_tensors = config.tokenizer.return_tensors,
                            return_token_type_ids = config.tokenizer.return_token_type_ids,
                            return_attention_mask = config.tokenizer.return_attention_mask)
    
    tokenized_highlighted_cells = []
    hard_relevance_label = torch.zeros((tokenized_input["input_ids"].shape[1]))
    for h_cell in highlighted_cells:
        x = tokenizer(answer = h_cell, add_special_tokens = False,
                            return_tensors = config.tokenizer.return_tensors,
                            return_attention_mask = config.tokenizer.return_attention_mask)["input_ids"].tolist()
        for ele in x[0]:
            hard_relevance_label[tokenized_input["input_ids"].squeeze() == ele] = 1
        
    hard_relevance_labels.append(hard_relevance_label)

In [ ]:
with open("datasets/wiki_tq_test_highlighted_cells_perturbed_table.pkl", "wb") as f:
    pickle.dump(hard_relevance_labels, f)

In [ ]:
with open("datasets/wiki_tq_test_highlighted_cells_perturbed_table_string.pkl", "wb") as f:
    pickle.dump(highlighted_cells, f)

### Inference on vanilla OmniTab

In [ ]:
import json
import pickle
from utils import process_config

In [ ]:
with open("configs/wiki_tq/tapex.json", "rb") as f:
    vanilla_qa_config = json.load(f)
vanilla_qa_config = process_config(vanilla_qa_config)

In [ ]:
with open("configs/wiki_tq_clustering_and_highlighting/tapex.json", "rb") as f:
    best_qa_config = json.load(f)
best_qa_config = process_config(best_qa_config)

In [ ]:
from src import BartModelForGenerativeQuestionAnswering

In [ ]:
from src import HighlightedCluBartModelForGenerativeQuestionAnswering

In [ ]:
model = BartModelForGenerativeQuestionAnswering(vanilla_qa_config)

In [ ]:
model = HighlightedCluBartModelForGenerativeQuestionAnswering(best_qa_config)

In [ ]:
import torch

In [ ]:
model.load_state_dict(torch.load("omnitab_best_ckpt/epoch=28.pt", map_location="cpu"))

In [ ]:
from data import WikiTQDataset

In [ ]:
from data import WikiTQHighlightedCellsDataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset("wikitablequestions")

In [ ]:
test_dataset = WikiTQHighlightedCellsDataset(dataset=dataset, config = best_qa_config, data_type = "test")

In [ ]:
with open("datasets/wiki_tq_test_perturbed_tables.pkl", "rb") as f:
    perturbed_tables_list = pickle.load(f)

In [ ]:
test_dataset.table = perturbed_tables_list

In [ ]:
with open("datasets/wiki_tq_test_highlighted_cells_perturbed_table.pkl", "rb") as f:
    hard_relevance_labels = pickle.load(f)

In [ ]:
test_dataset.highlighted_cells = hard_relevance_labels

In [ ]:
vanilla_qa_config.data.use_highlighted_cells = False
vanilla_qa_config.data.use_reason_in_output = False
vanilla_qa_config.data.use_reason_in_input = False


In [ ]:
inference_test_dataset = WikiTQDataset(dataset = dataset, config = vanilla_qa_config, data_type = "test")

In [ ]:
inference_test_dataset.table = perturbed_tables_list

In [ ]:
import os
import re
import sys
import argparse
import unicodedata
import numpy as np
import pandas as pd
from tqdm import tqdm
from codecs import open
from math import isnan, isinf
from easydict import EasyDict
from torch.utils.data import DataLoader
from abc import ABCMeta, abstractmethod

def normalize(x):

    if not isinstance(x, str):
        x = x.decode('utf8', errors='ignore')

    # Remove diacritics
    x = ''.join(c for c in unicodedata.normalize('NFKD', x)
                if unicodedata.category(c) != 'Mn')
    
    # Normalize quotes and dashes
    x = re.sub(r"[‘’´`]", "'", x)
    x = re.sub(r"[“”]", "\"", x)
    x = re.sub(r"[‐‑‒–—−]", "-", x)

    while True:
        
        old_x = x

        # Remove citations
        x = re.sub(r"((?<!^)\[[^\]]*\]|\[\d+\]|[•♦†‡*#+])*$", "", x.strip())
        
        # Remove details in parenthesis
        x = re.sub(r"(?<!^)( \([^)]*\))*$", "", x.strip())
        
        # Remove outermost quotation mark
        x = re.sub(r'^"([^"]*)"$', r'\1', x.strip())
        
        if x == old_x:
            break
    
    # Remove final '.'
    if x and x[-1] == '.':
        x = x[:-1]
    
    # Collapse whitespaces and convert to lower case
    x = re.sub(r'\s+', ' ', x, flags=re.U).lower().strip()
    
    return x


class Value(object):
    __metaclass__ = ABCMeta

    # Should be populated with the normalized string
    _normalized = None

    @abstractmethod
    def match(self, other):
        """Return True if the value matches the other value.

        Args:
            other (Value)
        Returns:
            a boolean
        """
        pass

    @property
    def normalized(self):
        return self._normalized


class StringValue(Value):

    def __init__(self, content):
        assert isinstance(content, str)
        self._normalized = normalize(content)
        self._hash = hash(self._normalized)

    def __eq__(self, other):
        return isinstance(other, StringValue) and self.normalized == other.normalized

    def __hash__(self):
        return self._hash

    def __str__(self):
        return 'S' + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        return self.normalized == other.normalized


class NumberValue(Value):

    def __init__(self, amount, original_string=None):
        assert isinstance(amount, (int, float))
        if abs(amount - round(amount)) < 1e-6:
            self._amount = int(amount)
        else:
            self._amount = float(amount)
        if not original_string:
            self._normalized = str(self._amount)
        else:
            self._normalized = normalize(original_string)
        self._hash = hash(self._amount)

    @property
    def amount(self):
        return self._amount

    def __eq__(self, other):
        return isinstance(other, NumberValue) and self.amount == other.amount

    def __hash__(self):
        return self._hash

    def __str__(self):
        return ('N(%f)' % self.amount) + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, NumberValue):
            return abs(self.amount - other.amount) < 1e-6
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a number.

        Return:
            the number (int or float) if successful; otherwise None.
        """
        try:
            return int(text)
        except:
            try:
                amount = float(text)
                assert not isnan(amount) and not isinf(amount)
                return amount
            except:
                return None


class DateValue(Value):

    def __init__(self, year, month, day, original_string=None):

        """Create a new DateValue. Placeholders are marked as -1."""
        assert isinstance(year, int)
        assert isinstance(month, int) and (month == -1 or 1 <= month <= 12)
        assert isinstance(day, int) and (day == -1 or 1 <= day <= 31)
        assert not (year == month == day == -1)
        
        self._year = year
        self._month = month
        self._day = day
        
        if not original_string:
            self._normalized = '{}-{}-{}'.format(
                year if year != -1 else 'xx',
                month if month != -1 else 'xx',
                day if day != '-1' else 'xx')
        else:
            self._normalized = normalize(original_string)
        
        self._hash = hash((self._year, self._month, self._day))

    @property
    def ymd(self):
        return (self._year, self._month, self._day)

    def __eq__(self, other):
        return isinstance(other, DateValue) and self.ymd == other.ymd

    def __hash__(self):
        return self._hash

    def __str__(self):
        return (('D(%d,%d,%d)' % (self._year, self._month, self._day))
                + str([self._normalized]))

    __repr__ = __str__

    def match(self, other):
        
        assert isinstance(other, Value)
        
        if self.normalized == other.normalized:
            return True
        
        if isinstance(other, DateValue):
            return self.ymd == other.ymd
        
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a date.

        Return:
            tuple (year, month, date) if successful; otherwise None.
        """
        try:
            ymd = text.lower().split('-')
            assert len(ymd) == 3
            year = -1 if ymd[0] in ('xx', 'xxxx') else int(ymd[0])
            month = -1 if ymd[1] == 'xx' else int(ymd[1])
            day = -1 if ymd[2] == 'xx' else int(ymd[2])
            assert not (year == month == day == -1)
            assert month == -1 or 1 <= month <= 12
            assert day == -1 or 1 <= day <= 31
            return (year, month, day)
        except:
            return None


def to_value(original_string, corenlp_value=None):
    """Convert the string to Value object.

    Args:
        original_string (basestring): Original string
        corenlp_value (basestring): Optional value returned from CoreNLP
    Returns:
        Value
    """

    if isinstance(original_string, Value):
        # Already a Value
        return original_string
    
    if not corenlp_value:
        corenlp_value = original_string
    
    # Number?
    amount = NumberValue.parse(corenlp_value)
    
    if amount is not None:
        return NumberValue(amount, original_string)
    
    # Date?
    ymd = DateValue.parse(corenlp_value)
    if ymd is not None:
        if ymd[1] == ymd[2] == -1:
            return NumberValue(ymd[0], original_string)
        else:
            return DateValue(ymd[0], ymd[1], ymd[2], original_string)
    
    # String.
    return StringValue(original_string)


def to_value_list(original_strings, corenlp_values=None):
    """Convert a list of strings to a list of Values

    Args:
        original_strings (list[basestring])
        corenlp_values (list[basestring or None])
    Returns:
        list[Value]
    """
    assert isinstance(original_strings, (list, tuple, set))
    if corenlp_values is not None:
        assert isinstance(corenlp_values, (list, tuple, set))
        assert len(original_strings) == len(corenlp_values)
        return list(set(to_value(x, y) for (x, y)
                        in zip(original_strings, corenlp_values)))
    else:
        return list(set(to_value(x) for x in original_strings))


def check_denotation(target_values, predicted_values):
    """Return True if the predicted denotation is correct.

    Args:
        target_values (list[Value])
        predicted_values (list[Value])
    Returns:
        bool
    """

    # Check size
    if len(target_values) != len(predicted_values):
        return False
    
    # Check items
    for target in target_values:
        if not any(target.match(pred) for pred in predicted_values):
            return False
    
    return True


def tsv_unescape(x):
    """Unescape strings in the TSV file.
    Escaped characters include:
        newline (0x10) -> backslash + n
        vertical bar (0x7C) -> backslash + p
        backslash (0x5C) -> backslash + backslash

    Args:
        x (str or unicode)
    Returns:
        a unicode
    """
    return x.replace(r'\n', '\n').replace(r'\p', '|').replace('\\\\', '\\')


def tsv_unescape_list(x):
    """Unescape a list in the TSV file.
    List items are joined with vertical bars (0x5C)

    Args:
        x (str or unicode)
    Returns:
        a list of unicodes
    """
    return [tsv_unescape(y) for y in x.split('|')]

In [ ]:
model.to("cuda:2")

In [ ]:
tokenizer = test_dataset.tokenizer

In [ ]:
def predict(index):

    input_ids, attention_mask, token_type_ids, decoder_input_ids, highlighted_cells, labels = test_dataset.__getitem__(index)
    output_ids = model.model.generate(input_ids = input_ids.unsqueeze(0).to("cuda:2"), max_new_tokens = vanilla_qa_config.tokenizer.output_max_length, 
                                    num_beams = 3, early_stopping = True, attention_mask = attention_mask.unsqueeze(0).to("cuda:2"),
                                    highlighted_cells=highlighted_cells.unsqueeze(0).to("cuda:2")).detach().cpu().squeeze()
    
    p = tokenizer.decode(output_ids, skip_special_tokens=True)
    a = tokenizer.decode(decoder_input_ids, skip_special_tokens = True)

    a = [x.strip().lower() for x in a.split(",")]
    p = [x.strip() for x in p.split(",")]

    pred = to_value_list(p)
    gold = to_value_list(a)

    verdict = check_denotation(gold, pred)

    return verdict

In [ ]:
correct = 0
total = 0

In [ ]:
for i in tqdm(range(len(test_dataset)), position=0, leave = True, total = len(test_dataset)):
    verdict = predict(i)
    if verdict:
        correct += 1
    total += 1

In [ ]:
correct / total

## Random row permutation

In [ ]:
from datasets import load_dataset
import torch
import torch.nn as nn
from transformers import AutoTokenizer
from src import BartModelForGenerativeQuestionAnswering
from tqdm import tqdm

In [ ]:
dataset = load_dataset("wikitablequestions")

In [ ]:
from data import WikiTQDataset
from utils import process_config
import json

In [ ]:
with open("configs/wiki_tq/tapex.json", "rb") as f:
    config = json.load(f)
config = process_config(config)

In [ ]:
test_dataset = WikiTQDataset(dataset=dataset, config=config, data_type="test")

In [ ]:
for i in range(len(test_dataset.table)):
    test_dataset.table[i] = test_dataset.table[i].sample(frac = 1)

In [ ]:
tokenizer = test_dataset.tokenizer

In [ ]:
model = BartModelForGenerativeQuestionAnswering(config=config)

In [ ]:
model.to("cuda:0")

In [ ]:
import os
import re
import sys
import argparse
import unicodedata
import numpy as np
import pandas as pd
from tqdm import tqdm
from codecs import open
from math import isnan, isinf
from easydict import EasyDict
from torch.utils.data import DataLoader
from abc import ABCMeta, abstractmethod

def normalize(x):

    if not isinstance(x, str):
        x = x.decode('utf8', errors='ignore')

    # Remove diacritics
    x = ''.join(c for c in unicodedata.normalize('NFKD', x)
                if unicodedata.category(c) != 'Mn')
    
    # Normalize quotes and dashes
    x = re.sub(r"[‘’´`]", "'", x)
    x = re.sub(r"[“”]", "\"", x)
    x = re.sub(r"[‐‑‒–—−]", "-", x)

    while True:
        
        old_x = x

        # Remove citations
        x = re.sub(r"((?<!^)\[[^\]]*\]|\[\d+\]|[•♦†‡*#+])*$", "", x.strip())
        
        # Remove details in parenthesis
        x = re.sub(r"(?<!^)( \([^)]*\))*$", "", x.strip())
        
        # Remove outermost quotation mark
        x = re.sub(r'^"([^"]*)"$', r'\1', x.strip())
        
        if x == old_x:
            break
    
    # Remove final '.'
    if x and x[-1] == '.':
        x = x[:-1]
    
    # Collapse whitespaces and convert to lower case
    x = re.sub(r'\s+', ' ', x, flags=re.U).lower().strip()
    
    return x


class Value(object):
    __metaclass__ = ABCMeta

    # Should be populated with the normalized string
    _normalized = None

    @abstractmethod
    def match(self, other):
        """Return True if the value matches the other value.

        Args:
            other (Value)
        Returns:
            a boolean
        """
        pass

    @property
    def normalized(self):
        return self._normalized


class StringValue(Value):

    def __init__(self, content):
        assert isinstance(content, str)
        self._normalized = normalize(content)
        self._hash = hash(self._normalized)

    def __eq__(self, other):
        return isinstance(other, StringValue) and self.normalized == other.normalized

    def __hash__(self):
        return self._hash

    def __str__(self):
        return 'S' + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        return self.normalized == other.normalized


class NumberValue(Value):

    def __init__(self, amount, original_string=None):
        assert isinstance(amount, (int, float))
        if abs(amount - round(amount)) < 1e-6:
            self._amount = int(amount)
        else:
            self._amount = float(amount)
        if not original_string:
            self._normalized = str(self._amount)
        else:
            self._normalized = normalize(original_string)
        self._hash = hash(self._amount)

    @property
    def amount(self):
        return self._amount

    def __eq__(self, other):
        return isinstance(other, NumberValue) and self.amount == other.amount

    def __hash__(self):
        return self._hash

    def __str__(self):
        return ('N(%f)' % self.amount) + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, NumberValue):
            return abs(self.amount - other.amount) < 1e-6
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a number.

        Return:
            the number (int or float) if successful; otherwise None.
        """
        try:
            return int(text)
        except:
            try:
                amount = float(text)
                assert not isnan(amount) and not isinf(amount)
                return amount
            except:
                return None


class DateValue(Value):

    def __init__(self, year, month, day, original_string=None):

        """Create a new DateValue. Placeholders are marked as -1."""
        assert isinstance(year, int)
        assert isinstance(month, int) and (month == -1 or 1 <= month <= 12)
        assert isinstance(day, int) and (day == -1 or 1 <= day <= 31)
        assert not (year == month == day == -1)
        
        self._year = year
        self._month = month
        self._day = day
        
        if not original_string:
            self._normalized = '{}-{}-{}'.format(
                year if year != -1 else 'xx',
                month if month != -1 else 'xx',
                day if day != '-1' else 'xx')
        else:
            self._normalized = normalize(original_string)
        
        self._hash = hash((self._year, self._month, self._day))

    @property
    def ymd(self):
        return (self._year, self._month, self._day)

    def __eq__(self, other):
        return isinstance(other, DateValue) and self.ymd == other.ymd

    def __hash__(self):
        return self._hash

    def __str__(self):
        return (('D(%d,%d,%d)' % (self._year, self._month, self._day))
                + str([self._normalized]))

    __repr__ = __str__

    def match(self, other):
        
        assert isinstance(other, Value)
        
        if self.normalized == other.normalized:
            return True
        
        if isinstance(other, DateValue):
            return self.ymd == other.ymd
        
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a date.

        Return:
            tuple (year, month, date) if successful; otherwise None.
        """
        try:
            ymd = text.lower().split('-')
            assert len(ymd) == 3
            year = -1 if ymd[0] in ('xx', 'xxxx') else int(ymd[0])
            month = -1 if ymd[1] == 'xx' else int(ymd[1])
            day = -1 if ymd[2] == 'xx' else int(ymd[2])
            assert not (year == month == day == -1)
            assert month == -1 or 1 <= month <= 12
            assert day == -1 or 1 <= day <= 31
            return (year, month, day)
        except:
            return None


def to_value(original_string, corenlp_value=None):
    """Convert the string to Value object.

    Args:
        original_string (basestring): Original string
        corenlp_value (basestring): Optional value returned from CoreNLP
    Returns:
        Value
    """

    if isinstance(original_string, Value):
        # Already a Value
        return original_string
    
    if not corenlp_value:
        corenlp_value = original_string
    
    # Number?
    amount = NumberValue.parse(corenlp_value)
    
    if amount is not None:
        return NumberValue(amount, original_string)
    
    # Date?
    ymd = DateValue.parse(corenlp_value)
    if ymd is not None:
        if ymd[1] == ymd[2] == -1:
            return NumberValue(ymd[0], original_string)
        else:
            return DateValue(ymd[0], ymd[1], ymd[2], original_string)
    
    # String.
    return StringValue(original_string)


def to_value_list(original_strings, corenlp_values=None):
    """Convert a list of strings to a list of Values

    Args:
        original_strings (list[basestring])
        corenlp_values (list[basestring or None])
    Returns:
        list[Value]
    """
    assert isinstance(original_strings, (list, tuple, set))
    if corenlp_values is not None:
        assert isinstance(corenlp_values, (list, tuple, set))
        assert len(original_strings) == len(corenlp_values)
        return list(set(to_value(x, y) for (x, y)
                        in zip(original_strings, corenlp_values)))
    else:
        return list(set(to_value(x) for x in original_strings))


def check_denotation(target_values, predicted_values):
    """Return True if the predicted denotation is correct.

    Args:
        target_values (list[Value])
        predicted_values (list[Value])
    Returns:
        bool
    """

    # Check size
    if len(target_values) != len(predicted_values):
        return False
    
    # Check items
    for target in target_values:
        if not any(target.match(pred) for pred in predicted_values):
            return False
    
    return True


def tsv_unescape(x):
    """Unescape strings in the TSV file.
    Escaped characters include:
        newline (0x10) -> backslash + n
        vertical bar (0x7C) -> backslash + p
        backslash (0x5C) -> backslash + backslash

    Args:
        x (str or unicode)
    Returns:
        a unicode
    """
    return x.replace(r'\n', '\n').replace(r'\p', '|').replace('\\\\', '\\')


def tsv_unescape_list(x):
    """Unescape a list in the TSV file.
    List items are joined with vertical bars (0x5C)

    Args:
        x (str or unicode)
    Returns:
        a list of unicodes
    """
    return [tsv_unescape(y) for y in x.split('|')]

In [ ]:
def get_verdict(index):

    input_ids, attention_mask, token_type_ids, decoder_input_ids, labels = test_dataset.__getitem__(index)
    output_ids = model.model.generate(input_ids = input_ids.unsqueeze(0).to("cuda:0"), max_new_tokens = config.tokenizer.output_max_length, 
                                    num_beams = 3, early_stopping = True, attention_mask = attention_mask.unsqueeze(0).to("cuda:0")).detach().cpu().squeeze()
    
    p = tokenizer.decode(output_ids, skip_special_tokens=True)
    a = tokenizer.decode(decoder_input_ids, skip_special_tokens = True)

    a = [x.strip().lower() for x in a.split(",")]
    p = [x.strip() for x in p.split(",")]

    pred = to_value_list(p)
    gold = to_value_list(a)

    verdict = check_denotation(gold, pred)

    return verdict

In [ ]:
correct = 0
total = 0

In [ ]:
for i in tqdm(range(len(test_dataset)), position = 0, leave = True):
    verdict = get_verdict(i)

    if verdict:
        correct += 1
    total += 1

In [ ]:
correct / total

# Radar Plot for WikiTQ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the categories (perturbation types)
categories = ['Row addition', 'Row permutation', 'Column permutation', 'Cell replacement', 'Row deletion']

# Define the data for the baseline and novel proposed models for each category

baseline_performance = 62.730
our_method_performance = 69.130

baseline_perturbed_performance = np.array([50.875, 57.804, 59.611, 56.256, 54.619])
our_method_perturbed_performance = np.array([61.205, 67.380, 68.116, 66.096, 64.778])

baseline_data = list(baseline_performance - baseline_perturbed_performance)
proposed_data = list(our_method_performance - our_method_perturbed_performance)

# baseline_data = [0.8, 0.6, 0.7, 0.5, 0.9]  # Example data for the baseline model
# proposed_data = [0.6, 0.8, 0.9, 0.7, 0.5]  # Example data for the novel proposed model

plt.figure(figsize=(6, 6))


# Number of categories
num_categories = len(categories)

# Create an array of angles (in radians) for each category
angles = np.linspace(0, 2 * np.pi, num_categories, endpoint=False).tolist()

# Make the plot circular
angles += angles[:1]

# Initialize the figure and axes
ax = plt.subplot(111, polar=True)

# Plot the data as a filled area
ax.fill(angles[:-1], baseline_data, 'b', alpha=0.2, c = "red")

# Set the labels for each axis
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)


# Number of categories
num_categories = len(categories)

# Create an array of angles (in radians) for each category
angles = np.linspace(0, 2 * np.pi, num_categories, endpoint=False).tolist()

# Make the plot circular
angles += angles[:1]

# Initialize the figure and axes
ax = plt.subplot(111, polar=True)

# Plot the data as a filled area
ax.fill(angles[:-1], proposed_data, 'b', alpha=0.1, c = "blue")

# Set the labels for each axis
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)

# Create a function to plot radar charts
# def plot_radar_chart(data, model_name):
#     # Number of categories
#     num_categories = len(categories)

#     # Create an array of angles (in radians) for each category
#     angles = np.linspace(0, 2 * np.pi, num_categories, endpoint=False).tolist()

#     # Make the plot circular
#     angles += angles[:1]

#     # Initialize the figure and axes
#     ax = plt.subplot(111, polar=True)

#     # Plot the data as a filled area
#     ax.fill(angles[:-1], data, 'b', alpha=0.2, c = "red")

#     # Set the labels for each axis
#     ax.set_xticks(angles[:-1])
#     ax.set_xticklabels(categories)

#     # Set the title
#     # plt.title(f'Performance Comparison ({model_name})')

# Plot radar charts for the baseline and novel proposed models
# plot_radar_chart(baseline_data, 'OmniTab')
# plot_radar_chart(proposed_data, 'Our Method')

# Show the radar charts
plt.legend(['Baseline Model', 'Our Method'], loc='lower left')
# plt.title(f'Drop in denotation accuracy on WikiTQ')
plt.savefig("wikitq_radar_plot.png")
plt.show()

# FeTaQA Radar Plot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the categories (perturbation types)
categories = ['Row addition', 'Row permutation', 'Column permutation', 'Cell replacement', 'Row deletion']

# Define the data for the baseline and novel proposed models for each category

baseline_performance = 34.890
our_method_performance = 40.532

baseline_perturbed_performance = np.array([27.353, 31.812, 30.732, 33.191, 31.740])
our_method_perturbed_performance = np.array([35.503, 38.898, 37.523, 39.591, 37.818])

baseline_data = list(baseline_performance - baseline_perturbed_performance)
proposed_data = list(our_method_performance - our_method_perturbed_performance)

# baseline_data = [0.8, 0.6, 0.7, 0.5, 0.9]  # Example data for the baseline model
# proposed_data = [0.6, 0.8, 0.9, 0.7, 0.5]  # Example data for the novel proposed model

plt.figure(figsize=(6, 6))


# Number of categories
num_categories = len(categories)

# Create an array of angles (in radians) for each category
angles = np.linspace(0, 2 * np.pi, num_categories, endpoint=False).tolist()

# Make the plot circular
angles += angles[:1]

# Initialize the figure and axes
ax = plt.subplot(111, polar=True)

# Plot the data as a filled area
ax.fill(angles[:-1], baseline_data, 'b', alpha=0.2, c = "red")

# Set the labels for each axis
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)


# Number of categories
num_categories = len(categories)

# Create an array of angles (in radians) for each category
angles = np.linspace(0, 2 * np.pi, num_categories, endpoint=False).tolist()

# Make the plot circular
angles += angles[:1]

# Initialize the figure and axes
ax = plt.subplot(111, polar=True)

# Plot the data as a filled area
ax.fill(angles[:-1], proposed_data, 'b', alpha=0.1, c = "blue")

# Set the labels for each axis
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)


# Create a function to plot radar charts
# def plot_radar_chart(data, model_name):
#     # Number of categories
#     num_categories = len(categories)

#     # Create an array of angles (in radians) for each category
#     angles = np.linspace(0, 2 * np.pi, num_categories, endpoint=False).tolist()

#     # Make the plot circular
#     angles += angles[:1]

#     # Initialize the figure and axes
#     ax = plt.subplot(111, polar=True)

#     # Plot the data as a filled area
#     ax.fill(angles[:-1], data, 'b', alpha=0.2, c = "green")

#     # Set the labels for each axis
#     ax.set_xticks(angles[:-1])
#     ax.set_xticklabels(categories)

#     # Set the title
#     # plt.title(f'Performance Comparison ({model_name})')

# # Plot radar charts for the baseline and novel proposed models
# plot_radar_chart(baseline_data, 'OmniTab')
# plot_radar_chart(proposed_data, 'Our Method')

# Show the radar charts
plt.legend(['Baseline Model', 'Novel Proposed Model'], loc='lower left')
# plt.title(f'Drop in denotation accuracy on WikiTQ')
plt.savefig("fetaqa_radar_plot.png")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the categories (perturbation types)
categories = ['Row addition', 'Row permutation', 'Column permutation', 'Cell replacement', 'Row deletion']

# Define the data for the baseline and novel proposed models for each category

baseline_performance = 87.911
our_method_performance = 89.180

baseline_perturbed_performance = np.array([81.325, 83.819, 83.367, 83.495, 84.263])
our_method_perturbed_performance = np.array([87.264, 88.161, 87.406, 88.228, 87.730])

baseline_data = list(baseline_performance - baseline_perturbed_performance)
proposed_data = list(our_method_performance - our_method_perturbed_performance)

# baseline_data = [0.8, 0.6, 0.7, 0.5, 0.9]  # Example data for the baseline model
# proposed_data = [0.6, 0.8, 0.9, 0.7, 0.5]  # Example data for the novel proposed model

plt.figure(figsize=(6, 6))


# Number of categories
num_categories = len(categories)

# Create an array of angles (in radians) for each category
angles = np.linspace(0, 2 * np.pi, num_categories, endpoint=False).tolist()

# Make the plot circular
angles += angles[:1]

# Initialize the figure and axes
ax = plt.subplot(111, polar=True)

# Plot the data as a filled area
ax.fill(angles[:-1], baseline_data, 'b', alpha=0.2, c = "red")

# Set the labels for each axis
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)


# Number of categories
num_categories = len(categories)

# Create an array of angles (in radians) for each category
angles = np.linspace(0, 2 * np.pi, num_categories, endpoint=False).tolist()

# Make the plot circular
angles += angles[:1]

# Initialize the figure and axes
ax = plt.subplot(111, polar=True)

# Plot the data as a filled area
ax.fill(angles[:-1], proposed_data, 'b', alpha=0.1, c = "blue")

# Set the labels for each axis
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)


# Create a function to plot radar charts
# def plot_radar_chart(data, model_name):
#     # Number of categories
#     num_categories = len(categories)

#     # Create an array of angles (in radians) for each category
#     angles = np.linspace(0, 2 * np.pi, num_categories, endpoint=False).tolist()

#     # Make the plot circular
#     angles += angles[:1]

#     # Initialize the figure and axes
#     ax = plt.subplot(111, polar=True)

#     # Plot the data as a filled area
#     ax.fill(angles[:-1], data, 'b', alpha=0.2, c = "green")

#     # Set the labels for each axis
#     ax.set_xticks(angles[:-1])
#     ax.set_xticklabels(categories)

#     # Set the title
#     # plt.title(f'Performance Comparison ({model_name})')

# # Plot radar charts for the baseline and novel proposed models
# plot_radar_chart(baseline_data, 'OmniTab')
# plot_radar_chart(proposed_data, 'Our Method')

# Show the radar charts
plt.legend(['Baseline Model', 'Novel Proposed Model'], loc='lower left')
# plt.title(f'Drop in denotation accuracy on WikiTQ')
plt.savefig("wikisql_radar_plot.png")
plt.show()

# Bart Plot for WikiTQ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the categories (perturbation types)
categories = ['Row addition', 'Row permutation', 'Column permutation', 'Cell replacement', 'Row deletion']

# Define the data for the baseline and novel proposed models for each category

baseline_performance = 62.730
our_method_performance = 69.130

baseline_perturbed_performance = np.array([50.875, 57.804, 59.611, 56.256, 54.619])
our_method_perturbed_performance = np.array([61.205, 67.380, 68.116, 66.096, 64.778])

baseline_data = list(baseline_performance - baseline_perturbed_performance)
proposed_data = list(our_method_performance - our_method_perturbed_performance)

# Number of categories
num_categories = len(categories)

# Create an array of indices for the categories
x = np.arange(num_categories)

# Width of each bar
bar_width = 0.2

# Create the bar plots for baseline and proposed models
plt.figure(figsize=(8, 5))
plt.bar(x - bar_width/2, baseline_data, bar_width, label='OmniTab', alpha=0.7)
plt.bar(x + bar_width/2, proposed_data, bar_width, label='Our Method', alpha=0.7)

# Set the x-axis labels
# plt.xlabel('Perturbation Types')
plt.xticks(x, categories)

# Set the y-axis label and title
plt.ylabel('Drop in performance')
# plt.title('Performance Comparison Across Perturbation Types')

# Add a legend
plt.legend()

# Show the bar plot
plt.tight_layout()

plt.savefig("wikitq_bar_plot.png")

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the categories (perturbation types)
categories = ['Row addition', 'Row permutation', 'Column permutation', 'Cell replacement', 'Row deletion']

# Define the data for the baseline and novel proposed models for each category

baseline_performance = 34.890
our_method_performance = 40.532

baseline_perturbed_performance = np.array([27.353, 31.812, 30.732, 33.191, 31.740])
our_method_perturbed_performance = np.array([35.503, 38.898, 37.523, 39.591, 37.818])

baseline_data = list(baseline_performance - baseline_perturbed_performance)
proposed_data = list(our_method_performance - our_method_perturbed_performance)

# Number of categories
num_categories = len(categories)

# Create an array of indices for the categories
x = np.arange(num_categories)

# Width of each bar
bar_width = 0.2

# Create the bar plots for baseline and proposed models
plt.figure(figsize=(8, 5))
plt.bar(x - bar_width/2, baseline_data, bar_width, label='OmniTab', alpha=0.7)
plt.bar(x + bar_width/2, proposed_data, bar_width, label='Our Method', alpha=0.7)

# Set the x-axis labels
# plt.xlabel('Perturbation Types')
plt.xticks(x, categories)

# Set the y-axis label and title
plt.ylabel('Drop in performance')
# plt.title('Performance Comparison Across Perturbation Types')

# Add a legend
plt.legend()

# Show the bar plot
plt.tight_layout()

plt.savefig("fetaqa_bar_plot.png")

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the categories (perturbation types)
categories = ['Row addition', 'Row permutation', 'Column permutation', 'Cell replacement', 'Row deletion']

# Define the data for the baseline and novel proposed models for each category

baseline_performance = 87.911
our_method_performance = 89.180

baseline_perturbed_performance = np.array([81.325, 83.819, 83.367, 83.495, 84.263])
our_method_perturbed_performance = np.array([87.264, 88.161, 87.406, 88.228, 87.730])

baseline_data = list(baseline_performance - baseline_perturbed_performance)
proposed_data = list(our_method_performance - our_method_perturbed_performance)

# Number of categories
num_categories = len(categories)

# Create an array of indices for the categories
x = np.arange(num_categories)

# Width of each bar
bar_width = 0.2

# Create the bar plots for baseline and proposed models
plt.figure(figsize=(8, 5))
plt.bar(x - bar_width/2, baseline_data, bar_width, label='OmniTab', alpha=0.7)
plt.bar(x + bar_width/2, proposed_data, bar_width, label='Our Method', alpha=0.7)

# Set the x-axis labels
# plt.xlabel('Perturbation Types')
plt.xticks(x, categories)

# Set the y-axis label and title
plt.ylabel('Drop in performance')
# plt.title('Performance Comparison Across Perturbation Types')

# Add a legend
plt.legend()

# Show the bar plot
plt.tight_layout()

plt.savefig("wikisql_bar_plot.png")

plt.show()


In [ ]:
from matplotlib import rcParams

rcParams['font.weight'] = 'bold'

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Define the categories (perturbation types)
categories = ['Row addition', 'Row permutation', 'Column permutation', 'Cell replacement', 'Row deletion']

# Define the data for the baseline and novel proposed models for each category

baseline_performance = 62.730
our_method_performance = 69.130

baseline_perturbed_performance = np.array([50.875, 57.804, 59.611, 56.256, 54.619])
our_method_perturbed_performance = np.array([61.205, 67.380, 68.116, 66.096, 64.778])

# baseline_data = list(baseline_performance - baseline_perturbed_performance)
baseline_data = list(baseline_perturbed_performance - baseline_performance)
# proposed_data = list(our_method_performance - our_method_perturbed_performance)
proposed_data = list(our_method_perturbed_performance - our_method_performance)

# Create a DataFrame for the data
data = pd.DataFrame({'Category': categories, 'Baseline Model': baseline_data, 'Proposed Method': proposed_data})

# Set a Seaborn style
sns.set_style("whitegrid")

# Create a color palette
colors = sns.color_palette("deep")

# Set the figure size
plt.figure(figsize=(7, 5))

# Create a bar plot
sns.barplot(x='Category', y='value', hue='variable', data=pd.melt(data, id_vars=['Category']), palette=colors, width=0.5)

# Set the x-axis label and title
# plt.xlabel('Perturbation Types', fontsize=12, loc="center")
plt.xlabel("Perturbation", fontsize = 10, weight = "bold")
plt.ylabel('Drop in Performance', fontsize=10, weight = "bold")
# plt.title('Performance Comparison Across Perturbation Types', fontsize=12)

# Customize legend
plt.legend(title='Method', labels=['OmniTab', 'Proposed Method'], fontsize=10)

legend = plt.gca().get_legend()
legend.legend_handles[0].set_color(colors[0])
legend.legend_handles[1].set_color(colors[1])

# Add grid lines
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Rotate x-axis labels for better readability
plt.xticks(rotation=15)

# Show the plot
plt.tight_layout()
plt.show()
